# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
%matplotlib inline

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection

`cv2.fillPoly()` for regions selection

`cv2.line()` to draw lines on an image given endpoints

`cv2.addWeighted()` to coadd / overlay two images

`cv2.cvtColor()` to grayscale or change color

`cv2.imwrite()` to output images to file

`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions for images

In [ ]:
def read_img(path):
    return mpimg.imread(path)

def write_img(img, path):
    if len(img.shape) > 2:
        # convert RGB to BGR (BGR2RGB works just fine)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # create directories if they do not exist
    os.makedirs(os.path.dirname(path), exist_ok=True)
    # save image
    cv2.imwrite(path, img)

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def hsv(img):
    """Applies the HSV transform"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    # Or use BGR2HSV if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

def hls(img):
    """Applies the HLS transform"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # Or use BGR2HLS if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2HLS)

def equalize_histogram(img):
    if len(img.shape) > 2:
        # source: https://stackoverflow.com/a/38312281
        img_yuv = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        # equalize the histogram of the Y channel
        img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
        # convert the YUV image back to RGB format
        return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)
    else:
        return cv2.equalizeHist(img)

def ensure_color(img):
    if len(img.shape) > 2:
        return img
    else:
        return cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.

    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)

    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    #filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).

    Think about things like separating line segments by their
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of
    the lines and extrapolate to the top and bottom of the lane.

    This function draws `lines` with `color` and `thickness`.
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.

    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

def hough_lines_img(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.

    Returns an image with hough lines drawn.
    """
    lines = hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, [lines])
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.

    `initial_img` should be the image before any processing.

    The result image is computed as follows:

    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Helper Functions for straights

In [ ]:
def get_straight_params_from_2points(x1, y1, x2, y2):
    m = (y2 - y1) / (x2 - x1)
    t = y1 - m * x1
    return (m, t)

def get_straight_params_from_line(line):
    x1, y1, x2, y2 = line
    return get_straight_params_from_2points(x1, y1, x2, y2)

def get_x_for_y_equals(straight_params, y):
    m, t = straight_params
    x = (y - t) / m
    return x

def reject_outlier_lines(lines, threshold_angle):
    if not lines or len(lines) <= 0:
        return []
    params_list = [get_straight_params_from_line(line) for line in lines]
    slopes = [m for (m, _) in params_list]
    # normalize angles for images (y-axis goes down -> negative of arctan)
    angles = [-np.arctan(m) % np.pi for m in slopes]
    median_angle = np.median(angles)
    # reject with threshold
    select_lines = abs(angles - median_angle) <= threshold_angle
    combined = zip(lines, select_lines)
    return [line for line, select in combined if select]

def approximate_line(lines, threshold_angle):
    lines = reject_outlier_lines(lines, threshold_angle)
    if not lines or len(lines) <= 0:
        return None
    y_values = [y1 for x1, y1, x2, y2 in lines] + [y2 for x1, y1, x2, y2 in lines]
    bottom = int(round(np.max(y_values)))
    top = int(round(np.min(y_values)))
    params_list = [get_straight_params_from_line(line) for line in lines]
    average_start_x = int(round(np.mean([get_x_for_y_equals(params, bottom) for params in params_list])))
    average_end_x = int(round(np.mean([get_x_for_y_equals(params, top) for params in params_list])))
    return (average_start_x, bottom, average_end_x, top)

def extrapolate_line_for_y_range(line, y_min, y_max):
    params = get_straight_params_from_line(line)
    x_min = int(round(get_x_for_y_equals(params, y_min)))
    x_max = int(round(get_x_for_y_equals(params, y_max)))
    return (x_min, y_min, x_max, y_max)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

## Build a Lane Finding Pipeline



In [ ]:
# write sample images for each pipeline step
write_sample_images_for_each_pipeline_step = False
def write_sample(img, name):
    global write_sample_images_for_each_pipeline_step
    if write_sample_images_for_each_pipeline_step:
        write_img(img, 'writeup_img/' + name + '.jpg')

In [ ]:
class Container(object):
    pass

def extract_lane_lines(img):
    # works fine like canny (works better on color images)
    kernel = np.ones((5,5), np.uint8)
    imgGradient = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel)

    # extracts thin lines quite good (works on both fine, color and grayscale images)
    #img = grayscale(img)
    kernel = np.ones((9,9), np.uint8)
    imgTopHat = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel)

    # combine both
    img = np.maximum(imgGradient, imgTopHat)

    # select the maximum channel of each pixel
    img[:,:,0][img[:,:,0] < img[:,:,1]] = 0
    img[:,:,0][img[:,:,0] < img[:,:,2]] = 0
    img[:,:,1][img[:,:,0] > 0] = 0
    img[:,:,2][img[:,:,0] > 0] = 0
    img = cv2.bitwise_or(cv2.bitwise_or(img[:,:,0], img[:,:,1]), img[:,:,2])

    # enhance detected areas
    kernel = np.ones((5,5), np.uint8)
    kernel = np.ones((3,3), np.uint8)
    img = cv2.dilate(img, kernel, iterations=2)
    #img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel) # does not work as good as expected

    # threshold
    threshold = 80
    img[img[:,:] < threshold] = 0
    img[img[:,:] > 0] = 255

    return img

def extract_lane_lines_test_hsv(img):
    # gaussian blur
    kernel_size = 5
    img = gaussian_blur(img, kernel_size)

    # transform to HSV
    img = hsv(img)

    """
    # return HSV
    return img
    """

    # color selection
    """
    red_threshold = 200
    green_threshold = 150
    blue_threshold = 0
    rgb_threshold = [red_threshold, green_threshold, blue_threshold]
    thresholds = (img[:,:,0] < rgb_threshold[0]) \
                | (img[:,:,1] < rgb_threshold[1]) \
                | (img[:,:,2] < rgb_threshold[2])
    """
    hue_value = img[:,:,0]
    sat_value = img[:,:,1]
    val_value = img[:,:,2]
    selections_yellow = \
        (hue_value <= 35) & (hue_value >= 18) \
        & (sat_value >= 70) \
        & (val_value >= 100) #& (val_value >= 18)
    selections_white = \
        (sat_value <= 80) \
        & (val_value >= 150) #& (val_value >= 18)
    selections = selections_yellow | selections_white
    thresholds = selections ^ True
    img = img[:,:,0]
    img[selection] = 255
    img[thresholds] = 0

    return img

def detect_edges(img):
    img = gaussian_blur(img, 5)
    img = canny(img, 100, 200)
    return img

def get_roi(width, height):
    """
    "                                            top_witdh
    "                                  |----------------------------|
    "
    "                                         top_width_inner
    "                                            |--------|
    "
    "                     XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
    "                     X                                                      X
    "                     X                                                      X
    "                     X                                                      X
    "                     X                                                      X
    "                     X                                                      X
    "                 ~   X             ----------------------------             X
    "             top |   X            /                            \            X
    "              ~  |   X           /           --------           \           X
    "              |  |   X          /           /        \           \          X
    "    top_inner |  |   X         /           /          \           \         X
    "              |  |   X        /           /            \           \        X
    "              |  |   X       /           /              \           \       X
    "           ~  |  |   X       ------------                ------------       X
    "    bottom |  |  |   X                                                      X
    "           |  |  |   X                                                      X
    "           ~  ~  ~   XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
    "
    "                            |------------|              |------------|
    "                             bottom_width                bottom_width
    "
    "                                         |--------------|
    "                                        bottom_width_inner
    """
    roi = Container()
    roi.top = int(round(height * 0.4))
    roi.top_inner = int(round(height * 0.3))
    roi.bottom = int(round(height * 0.0))
    roi.top_width = int(round(width * 0.15))
    roi.top_width_inner = int(round(width * 0.0))
    roi.bottom_width = int(round(width * 0.3))
    roi.bottom_width_inner = int(round(width * 0.4))

    roi.vertices = [ \
        (int(round((width - 2 * roi.bottom_width - roi.bottom_width_inner) / 2)), height - roi.bottom), \
        (int(round((width - roi.top_width) / 2)), height - roi.top), \
        (int(round(width - (width - roi.top_width) / 2)), height - roi.top), \
        (int(round(width - (width - 2 * roi.bottom_width - roi.bottom_width_inner) / 2)), height - roi.bottom), \
        (int(round(width - (width - 2 * roi.bottom_width - roi.bottom_width_inner) / 2 - roi.bottom_width)), height - roi.bottom), \
        (int(round(width - (width - roi.top_width_inner) / 2)), height - roi.top_inner), \
        (int(round((width - roi.top_width_inner) / 2)), height - roi.top_inner), \
        (int(round((width - 2 * roi.bottom_width - roi.bottom_width_inner) / 2 + roi.bottom_width)), height - roi.bottom) \
    ]
    roi.vertices_lines = [(x1, y1, x2, y2) for (x1, y1), (x2, y2) in zip(roi.vertices, roi.vertices[1:] + [roi.vertices[0]])] # make lines
    return roi

def detect_hough_lines(img):
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 9 # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 12 # minimum number of pixels making up a line
    max_line_gap = 5 # maximum gap in pixels between connectable line segments
    lines = hough_lines(img, rho, theta, threshold, min_line_length, max_line_gap)

    detected_lines = []
    for polyLine in lines:
        for line in polyLine:
            detected_lines.append(line)
    return detected_lines

def filter_lines(lines, width, height, roi, side):
    """
    side can be ('left', 'right', 'both')
    """
    bottom_region_left = int(round((width - roi.bottom_width_inner - roi.bottom_width) / 2))
    bottom_region_right = width - bottom_region_left
    max_delta_bottom = int(round(roi.bottom_width / 2))
    top_region = int(round(width / 2))
    max_delta_top = int(round(roi.top_width / 2))
    """
    img_start_filter_roi = np.zeros_like(img_orig)
    draw_lines(img_start_filter_roi, [[(bottom_region_left - max_delta_bottom, height - roi.bottom, bottom_region_left + max_delta_bottom, height - roi.bottom)]], thickness=5)
    draw_lines(img_start_filter_roi, [[(bottom_region_right - max_delta_bottom, height - roi.bottom, bottom_region_right + max_delta_bottom, height - roi.bottom)]], thickness=5)
    draw_lines(img_start_filter_roi, [[(top_region - max_delta_top, height - roi.top, top_region + max_delta_top, height - roi.top)]], thickness=5)
    """
    filtered_lines = []
    for line in lines:
        params = get_straight_params_from_line(line)
        x_top = get_x_for_y_equals(params, height - roi.top)
        delta_top = abs(x_top - top_region)
        if delta_top <= max_delta_top:
            x_bottom = get_x_for_y_equals(params, height - roi.bottom)
            if not(side == 'right'):
                # is left lane?
                delta_left_bottom = abs(x_bottom - bottom_region_left)
                if delta_left_bottom <= max_delta_bottom:
                    filtered_lines.append(line)
            if not(side == 'left'):
                # is right lane?
                delta_right_bottom = abs(x_bottom - bottom_region_right)
                if delta_right_bottom <= max_delta_bottom:
                    filtered_lines.append(line)
    return filtered_lines

In [ ]:
def detect_lane_lines(img):
    img_orig = img.copy()
    width = img.shape[1]
    height = img.shape[0]

    # histogram equalization weakens the lane lines detection
    #img = equalize_histogram(img)

    img = extract_lane_lines(img)
    write_sample(img, 'extractedLaneLines')
    #img = extract_lane_lines_test_hsv(img)
    #write_sample(img, 'extractedLaneLinesHSV')

    img = detect_edges(img)
    write_sample(img, 'detectedEdges')

    # region of interest
    roi = get_roi(width, height)
    img = region_of_interest(img, np.array([roi.vertices], dtype=np.int32))
    if write_sample_images_for_each_pipeline_step:
        img_roi = np.zeros_like(img_orig)
        draw_lines(img_roi, [roi.vertices_lines], thickness=5)
        img_roi[:,:,1:2] = 0 # only keep red channel
        img_roi = weighted_img(img_orig, img_roi, 1, .3)
        write_sample(img_roi, 'roi')

    # hough transform
    lines = detect_hough_lines(img)
    if write_sample_images_for_each_pipeline_step:
        img_hough = np.zeros_like(img_orig)
        draw_lines(img_hough, [lines], thickness=5)
        img_hough[:,:,1:2] = 0 # only keep red channel
        img_hough = weighted_img(img_orig, img_hough, 1, .3)
        write_sample(img_hough, 'houghLines')

    # filter lines
    left_lines = filter_lines(lines, width, height, roi, 'left')
    right_lines = filter_lines(lines, width, height, roi, 'right')
    if write_sample_images_for_each_pipeline_step:
        img_rl = np.zeros_like(img_orig)
        draw_lines(img_rl, [left_lines + right_lines], thickness=5)
        img_rl[:,:,1:2] = 0 # only keep red channel
        img_rl = weighted_img(img_orig, img_rl, 1, .3)
        write_sample(img_rl, 'leftRightLines')

    # approximate two lines
    threshold_angle = np.pi / 180 * 5
    lane_lines = []
    if write_sample_images_for_each_pipeline_step:
        img_rejected = np.zeros_like(img_orig)
        draw_lines(img_rejected, [reject_outlier_lines(left_lines, threshold_angle) + reject_outlier_lines(right_lines, threshold_angle)], thickness=5)
        img_rejected[:,:,1:2] = 0 # only keep red channel
        img_rejected = weighted_img(img_orig, img_rejected, 1, .3)
        write_sample(img_rejected, 'rejectOutlierLines')
    line = approximate_line(left_lines, threshold_angle)
    if line:
        lane_lines.append(line)
    line = approximate_line(right_lines, threshold_angle)
    if line:
        lane_lines.append(line)
    if write_sample_images_for_each_pipeline_step:
        img_approx = np.zeros_like(img_orig)
        draw_lines(img_approx, [lane_lines], thickness=10)
        img_approx[:,:,1:2] = 0 # only keep red channel
        img_approx = weighted_img(img_orig, img_approx, 1, .3)
        write_sample(img_approx, 'approximatedLines')

    # extrapolate lane lines
    lane_lines = [extrapolate_line_for_y_range(line, height - roi.top, height - roi.bottom) for line in lane_lines]

    # draw lines
    img = np.zeros_like(img_orig)
    draw_lines(img, [lane_lines], thickness=10)

    # combine
    img = weighted_img(img_orig, img)

    # return result
    write_sample(img, 'finalLaneLines')
    return img

    # return result for debugging
    #imgBeforeCanny = ensure_color(imgBeforeCanny)
    #imgBeforeCanny[:,:,0] = 0
    #imgBeforeCanny[:,:,2] = 0
    #return weighted_img(weighted_img(img_orig, img,1,.3), ensure_color(imgBeforeCanny))
    #imgAfterCanny = ensure_color(imgAfterCanny)
    #imgAfterCanny[:,:,0] = 0
    #imgAfterCanny[:,:,2] = 0
    #return weighted_img(weighted_img(img_orig, img,1,.3), ensure_color(imgAfterCanny))
    #return weighted_img(weighted_img(img_orig, img,1,.3), ensure_color(img_start_filter_roi))
    #return weighted_img(weighted_img(img_orig, img,1,.3), ensure_color(imgBeforeHough))
    #return weighted_img(img_orig, img,1,.3)

In [ ]:
# enable samples
write_sample_images_for_each_pipeline_step = True
# read image
img_sample = read_img('test_images/71.jpg')
# do processing
detect_lane_lines(img_sample)
# reset/disable samples
write_sample_images_for_each_pipeline_step = False

In [ ]:
# nowStr for the filenames of the outputs
import datetime
now = datetime.datetime.now()
nowStr = now.strftime("%Y-%m-%d_%H-%M-%S")
nowStr

def get_output_file(path):
    dir = os.path.dirname(path)
    file = os.path.basename(path)
    fileName = ('.').join(file.split('.')[:-1])
    fileExt = '.' + file.split('.')[-1]
    return os.path.join(os.getcwd(), dir + "_output", fileName + '_' + nowStr + fileExt)

# get a list of test images
imgDir = "test_images/"
imgPaths = [os.path.join(imgDir, path) for path in os.listdir(imgDir)]
imgPaths.sort()
imgPaths

# run the test images
for imgPath in imgPaths:
    # read image
    img = read_img(imgPath)
    # do processing
    img = detect_lane_lines(img)
    # plot image in separate figure
    plt.figure()
    plt.title(imgPath)
    if len(img.shape) > 2:
        plt.imshow(img)
    else:
        plt.imshow(img, cmap='gray')
    # get save path for image
    outputFile = get_output_file(imgPath)
    # write image
    write_img(img, outputFile)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [ ]:
def process_image(img):
    img = detect_lane_lines(img)
    img = ensure_color(img)
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    return img

Let's try the one with the solid white lane on the right first ...

In [ ]:
# ensure output directory exists
os.makedirs('test_videos_output', exist_ok=True)

In [ ]:
white_output = 'test_videos_output/solidWhiteRight_' + nowStr + '.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft_' + nowStr + '.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge_' + nowStr + '.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))